# 1) Import Libraries

In [ ]:
import numpy as np
import pandas as pd

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import operator
from sklearn.model_selection import KFold

# 2) Load data

In [ ]:
train = pd.read_csv('../input/train.tsv', sep='\t')
test = pd.read_csv('../input/test.tsv', sep='\t')

In [ ]:
train.describe(include="all")

# 3) Data Analysis

In [ ]:
#get a list of the features within the dataset
print(train.columns)

In [ ]:
#see a sample of the dataset to get an idea of the variables
train.sample(5)

Numerical Features: train_id, item_condition_id, price, shipping

Categorical Features: name, category_name, brand_name, item_description

In [ ]:
#check for any other unusable values
print(pd.isnull(train).sum())

Around 45% of brand_name is missing. 

# 4) Data Visualization

## Name

In [ ]:
from wordcloud import WordCloud

In [ ]:
wordcloud = WordCloud(width = 1000, height = 500).generate(' '.join(train['name']))
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

## item_condition_id feature

In [ ]:
#draw a bar plot of item_condition_id by price
sns.barplot(x="item_condition_id", y="price", data=train)

## shipping feature

In [ ]:
#draw a bar plot of shipping by price
sns.barplot(x="shipping", y="price", data=train)

# 4) Feature Engineering

## Name

In [ ]:
#Add new feature with length of the name
train['name_len_words'] = train['name'].apply(lambda x: len(x.split()))
test['name_len_words'] = test['name'].apply(lambda x : len(x.split()))

train['name_len_chars'] = train['name'].apply(lambda x: len(x))
test['name_len_chars'] = test['name'].apply(lambda x : len(x))

In [ ]:
train = train.drop('name', axis = 1)
test = test.drop('name', axis = 1)

## Category Name

In [ ]:
#print category name and its frequency
category_dict = train['category_name'].value_counts().to_dict()

In [ ]:
sorted_category_dict = sorted(category_dict.items(), key=operator.itemgetter(1))

In [ ]:
sorted_category_dict[::-1]
sorted_category_list = [v[0] for v in sorted_category_dict]

In [ ]:
train['category_name'] = train['category_name'].fillna(pd.Series(np.random.choice(sorted_category_list[0:100], size=len(train.index))))
test['category_name'] = test['category_name'].fillna(pd.Series(np.random.choice(sorted_category_list[0:100], size=len(test.index))))

In [ ]:
#check for any other unusable values
print(pd.isnull(train).sum())

In [ ]:
#Add column with number of elements in category_name
train['category_name_elements'] = train['category_name'].apply(lambda x: len(x))
test['category_name_elements'] = test['category_name'].apply(lambda x: len(x))

In [ ]:
train.sample(5)

In [ ]:
train = train.drop('category_name', axis = 1)
test = test.drop('category_name', axis = 1)

## brand_name feature

In [ ]:
# Drop brand name column as more than half of values are 'NA'
train = train.drop('brand_name', axis=1)
test = test.drop('brand_name', axis=1)

## item_description feature

In [ ]:
#Only 4 elements are None, so fill them with some random string
train['item_description'] = train['item_description'].fillna('Description Not available')
test['item_description'] = test['item_description'].fillna('Description Not available')

In [ ]:
#Length of description and number of words as new parameters
train['item_desc_words'] = train['item_description'].apply(lambda x: len(x.split()))
test['item_desc_words'] = test['item_description'].apply(lambda x: len(x.split()))

train['item_desc_chars'] = train['item_description'].apply(lambda x: len(x))
test['item_desc_chars'] = test['item_description'].apply(lambda x: len(x))

In [ ]:
train.sample(5)

In [ ]:
train = train.drop('item_description', axis=1)
test = test.drop('item_description', axis=1)

In [ ]:
train.sample(5)

# 5) Divide data into train and valid

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.drop(['train_id', 'price'], axis=1), train['price'], test_size=0.10, random_state=42)

In [ ]:
X_train.shape

# 6) RMSE

In [ ]:
def calculate_RMSE(y, y_pred):
    k = len(y)
    s = 0
    for i in range(k):
        s += np.square((np.log10(y_pred[i] + 1) - np.log10(y[i] + 1)))
    s = s/k
    return np.sqrt(s)

# 7) Regression Algorithms

In [ ]:
from sklearn import linear_model

In [ ]:
clf = linear_model.BayesianRidge()

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_test)

In [ ]:
print (calculate_RMSE(y_test.tolist(), preds))

In [ ]:
target = test['test_id']
predictions = clf.predict(test.drop(['test_id'], axis=1))
predictions = predictions/5.0

In [ ]:
submission = pd.DataFrame()
submission['test_id'] = target
submission['price'] = predictions

In [ ]:
submission.to_csv('submission.csv', index = False)

## XGB

In [ ]:
import xgboost as xgb

In [ ]:
xgb_params= {  
            'eta': 0.7,
            'max_depth': 12,
            'objective':'reg:linear',
            'eval_metric':'rmse',
            'silent': 1
}

In [ ]:
kf = KFold(n_splits = 5, random_state = 1, shuffle = True)

In [ ]:
train_matrix = xgb.DMatrix(X_train, y_train)

In [ ]:
validation_matrix = xgb.DMatrix(X_test, y_test)

In [ ]:
evallist  = [(validation_matrix,'validation')]

In [ ]:
model = xgb.train(xgb_params, train_matrix, 10, evallist, verbose_eval=2)

In [ ]:
preds = model.predict(xgb.DMatrix(test.drop(['test_id'], axis=1)), ntree_limit=model.best_ntree_limit);

In [ ]:
print (preds)

In [ ]:
submission = pd.DataFrame()
submission['test_id'] = test['test_id']
submission['price'] = preds

In [ ]:
submission.to_csv('submission.csv', index = False)

## RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
regr = RandomForestRegressor(max_depth=2, random_state=0)

In [ ]:
regr.fit(train.drop(['train_id', 'price'], axis=1), train['price'])

In [ ]:
preds = regr.predict(test.drop(['test_id'], axis=1))

In [ ]:
print (calculate_RMSE(y_test.tolist(), preds))

In [ ]:
submission = pd.DataFrame()
submission['test_id'] = test['test_id']
submission['price'] = preds

In [ ]:
submission.to_csv('submission.csv', index = False)